In [1]:
# 1-) Yeni Ürünlerin sektörlerini bulmak

import pandas as pd

# CSV dosyalarını yükleyelim
df_2007 = pd.read_csv(r"C:\Users\oguzo\Desktop\BACI\BACI_HS07_V202401\product_codes_HS07_V202401.csv")
df_2012 = pd.read_csv(r"C:\Users\oguzo\Desktop\BACI\BACI_HS12_V202401\product_codes_HS12_V202401.csv")
df_2017 = pd.read_csv(r"C:\Users\oguzo\Desktop\BACI\BACI_HS17_V202401\product_codes_HS17_V202401.csv")
df_2022 = pd.read_csv(r"C:\Users\oguzo\Desktop\BACI\BACI_HS22_V202401\product_codes_HS22_V202401.csv")

# 2007, 2012, 2017 ve 2022 yıllarına ait product code'ları set yap
products_2007 = set(df_2007['code'])
products_2012 = set(df_2012['code'])
products_2017 = set(df_2017['code'])
products_2022 = set(df_2022['code'])

# 2012, 2017 ve 2022'de yeni eklenen ürünler
new_products_2012 = products_2012 - products_2007
new_products_2012_df = df_2012[df_2012['code'].isin(new_products_2012)].copy()
new_products_2017 = products_2017 - products_2012
new_products_2017_df = df_2017[df_2017['code'].isin(new_products_2017)].copy()
new_products_2022 = products_2022 - products_2017
new_products_2022_df = df_2022[df_2022['code'].isin(new_products_2022)].copy()

# Her yıla ait veri setine "Yıl" sütunu ekleyelim
new_products_2012_df['Yıl'] = 2012
new_products_2017_df['Yıl'] = 2017
new_products_2022_df['Yıl'] = 2022

# Üç yıldaki yeni ürünleri aynı data frame içine al
combined_new_products = pd.concat([new_products_2022_df, new_products_2017_df, new_products_2012_df], ignore_index=True)

# 6 basamaklı HS kodlarını 10000'e bölerek 2 basamaklı sektörlerini bul
combined_new_products['2 Basamaklı HS Kodu'] = combined_new_products['code'] // 10000

# Atlas verisini çağırıp 2 basamaklı HS kodlarının isimlerini çekelim
combined_atlas_df = pd.read_excel('0-) Atlastan Çekilmiş HS92 İsimleri 1 Basamak İçin.xlsx')
combined_atlas_df['Code_2'] = combined_atlas_df['2 Basamaklı HS Kodları'].str.extract(r'\((\d{2})')
combined_atlas_df['Code_1'] = combined_atlas_df['1 Basamaklı HS Kodları'].str.extract(r'\((\d{1})')

# Bir sözlük oluşturarak 2 basamaklı HS kodları ve isimleri arasında bir map oluşturalım
code_2_to_name_mapping = dict(zip(combined_atlas_df['Code_2'], combined_atlas_df['2 Basamaklı HS İsimleri']))
code_2_to_code_1_mapping = dict(zip(combined_atlas_df['Code_2'], combined_atlas_df['Code_1']))
code_1_to_name_mapping = dict(zip(combined_atlas_df['Code_1'], combined_atlas_df['1 Basamaklı HS İsimleri']))

# 2 basamaklı HS kodlarını ve bunlara karşılık gelen 1 basamaklı kodları ve isimleri ekleyelim
combined_new_products['2 Basamaklı HS İsmi'] = combined_new_products['2 Basamaklı HS Kodu'].astype(str).str.zfill(2).map(code_2_to_name_mapping)
combined_new_products['1 Basamaklı HS Kodu'] = combined_new_products['2 Basamaklı HS Kodu'].astype(str).str.zfill(2).map(code_2_to_code_1_mapping)
combined_new_products['1 Basamaklı HS İsmi'] = combined_new_products['1 Basamaklı HS Kodu'].map(code_1_to_name_mapping)

# 6 basamaklı HS kodunu ve detaylı sektör ismini zaten koruyoruz
combined_new_products = combined_new_products.rename(columns={'description': 'Detaylı Sektör İsmi', 'code': '6 Basamaklı HS Kodu'})  

# "Yıl" sütununu en başa taşıyalım
cols = ['Yıl'] + [col for col in combined_new_products.columns if col != 'Yıl']
combined_new_products = combined_new_products[cols]

# Excele kaydet
combined_new_products.to_excel("1-) 2012, 2017 ve 2022'deki yeni ürünlerin sektörleri.xlsx")

In [48]:
import pandas as pd

# Yıllar ve HS versiyonları
yillar = list(range(2012, 2022))
hs_versiyonlari = {year: 'HS12' if year <= 2016 else 'HS17' for year in yillar}

# Dosya yolu formatı
dosya_yolu_format = r"C:\Users\oguzo\Desktop\BACI\BACI_{hs_version}_V202401\BACI_{hs_version}_Y{year}_V202401.csv"

# Yeni ürünlerin bulunduğu Excel dosyasını yükle
new_products_df = pd.read_excel("1-) 2012, 2017 ve 2022'deki yeni ürünlerin sektörleri.xlsx")

# 2012 ve 2017'ye ait yeni ürünleri ayır
new_products_2012_df = new_products_df[new_products_df['Yıl'] == 2012]
new_products_2017_df = new_products_df[new_products_df['Yıl'] == 2017]

# Sonuçları tutmak için bir liste
toplam_oranlar_list = []

# Her yıl için döngü
for year in yillar:
    hs_version = hs_versiyonlari[year] 
    dosya_yolu = dosya_yolu_format.format(hs_version=hs_version, year=year)  
    
    # Veriyi okuma
    try:
        df = pd.read_csv(dosya_yolu)

        # Toplam ticaret hacmini hesapla
        toplam_ticaret_hacmi = df['v'].sum()

        # 2012-2016 için 2012'deki yeni ürünler, 2017-2021 için 2017'deki yeni ürünler kullan
        if year <= 2016:
            yeni_urunler_ticaret_hacmi = df[df['k'].isin(new_products_2012_df['6 Basamaklı HS Kodu'])]['v'].sum()
        else:
            yeni_urunler_ticaret_hacmi = df[df['k'].isin(new_products_2017_df['6 Basamaklı HS Kodu'])]['v'].sum()

        # Yıl, toplam ticaret hacmi ve yeni ürün ticaret hacmi oranını hesapla
        oran = yeni_urunler_ticaret_hacmi / toplam_ticaret_hacmi if toplam_ticaret_hacmi > 0 else 0
        toplam_oranlar_list.append({'Yıl': year, 'Toplam Ticaret Hacmi': toplam_ticaret_hacmi, 'Yeni Ürünler Ticaret Hacmi': yeni_urunler_ticaret_hacmi, 'Oran': oran})

    except FileNotFoundError:
        print(f"{year} yılına ait dosya bulunamadı: {dosya_yolu}")
        continue

# Sonuçları DataFrame'e dönüştürme
toplam_oranlar_df = pd.DataFrame(toplam_oranlar_list)

# Sonuçları Excel dosyasına kaydetme
output_file_path = "a.xlsx"
toplam_oranlar_df.to_excel(output_file_path, index=False)

print(f"Dosya kaydedildi: {output_file_path}")


Dosya kaydedildi: a.xlsx


In [47]:
# 2-) Yeni ürünlerin sektörlerdeki sayıları

import pandas as pd

# Dosyayı yükle
new_products_df = pd.read_excel("1-) 2012, 2017 ve 2022'deki yeni ürünlerin sektörleri.xlsx")

# Her yıl için sektör isimlerine ve 2 basamaklı HS koduna göre grupla
sector_grouping_by_year = new_products_df.groupby(['Yıl', 'Sektör İsmi', '2 Basamaklı HS Kodu']).size().reset_index(name='Sayı')

# Sonuçları Excel dosyasına kaydet
sector_grouping_by_year.to_excel('2-) 2012, 2017 ve 2022\'deki yeni ürünlerin sektörlerdeki sayıları.xlsx', index=False)

KeyError: 'Sektör İsmi'

In [11]:
# 3-) Yeni ürünlerin ticaret hacmindeki toplam oranları 2012-2021

import pandas as pd

# Yıllar ve ilgili HS sürümleri 
yillar = list(range(2012, 2022))
hs_versiyonlari = {year: 'HS12' if year <= 2016 else 'HS17' for year in yillar}

# Dosya yolu formatı 
dosya_yolu_format = r"C:\Users\oguzo\Desktop\BACI\BACI_{hs_version}_V202401\BACI_{hs_version}_Y{year}_V202401.csv"

# Yeni ürünlerin bulunduğu Excel dosyasını yükle
new_products_df = pd.read_excel('1-) 2012, 2017 ve 2022\'deki yeni ürünlerin detaylı ve genel sektörleri.xlsx')
new_products_df['Yıl'] = new_products_df['Yıl'].ffill()

# 2012 ve 2017'ye ait yeni ürünleri ayır
new_products_2012_df = new_products_df[new_products_df['Yıl'] == 2012]
new_products_2017_df = new_products_df[new_products_df['Yıl'] == 2017]
new_products_2022_df = new_products_df[new_products_df['Yıl'] == 2022]

# Sonuçları tutmak için bir liste
toplam_oranlar_list = []

# Her yıl için döngü
for year in yillar:
    hs_version = hs_versiyonlari[year] 
    dosya_yolu = dosya_yolu_format.format(hs_version=hs_version, year=year)  
    
    # Veriyi okuma
    df = pd.read_csv(dosya_yolu)
    
    # Toplam ticaret hacmini hesapla
    toplam_ticaret_hacmi = df['v'].sum()

    # 2012-2016 için 2012'deki yeni ürünler, 2017-2021 için 2017'deki yeni ürünler kullan
    if year <= 2016:
        yeni_urunler_ticaret_hacmi = df[df['k'].isin(new_products_2012_df['6 Basamaklı HS Kodu'])]['v'].sum()
    else:
        yeni_urunler_ticaret_hacmi = df[df['k'].isin(new_products_2017_df['6 Basamaklı HS Kodu'])]['v'].sum()
    
    # Yeni ürünlerin ticaret hacminin toplam ticaret hacmine oranı
    yeni_urunler_orani = (yeni_urunler_ticaret_hacmi / toplam_ticaret_hacmi) * 100
    
    # Sonuçları listeye ekle
    toplam_oranlar_list.append({'Yıl': year, 'Yeni Ürünlerin Oranı (%)': yeni_urunler_orani})

# Sonuçları DataFrame'e çevir ve excel olarak kaydet
toplam_oranlar = pd.DataFrame(toplam_oranlar_list)

toplam_oranlar.to_excel("3-) Yeni ürünlerin ticaret hacmindeki toplam oranları 2012-2021.xlsx", index=False)

In [5]:
# 4-) YAPILACAK
# Yeni ürünlerin ticaret hacmindeki sektörel bazlı oranları 2012-2021

import pandas as pd

# Yıllar ve ilgili HS sürümleri 
yillar = list(range(2012, 2022))
hs_versiyonlari = {year: 'HS12' if year <= 2016 else 'HS17' for year in yillar}

# Dosya yolu formatı 
dosya_yolu_format = r"C:\Users\oguzo\Desktop\BACI\BACI_{hs_version}_V202401\BACI_{hs_version}_Y{year}_V202401.csv"

# Yeni ürünlerin bulunduğu Excel dosyasını yükle
new_products_df = pd.read_excel('2012, 2017 ve 2022\'deki yeni ürünlerin detaylı ve genel sektörleri.xlsx')
new_products_df['Yıl'] = new_products_df['Yıl'].ffill()

# 2012 ve 2017'ye ait yeni ürünleri ayır
new_products_2012_df = new_products_df[new_products_df['Yıl'] == 2012]
new_products_2017_df = new_products_df[new_products_df['Yıl'] == 2017]
new_products_2022_df = new_products_df[new_products_df['Yıl'] == 2022]



,Yıl,6 Basamaklı HS Kodu,Detaylı Sektör İsmi,2 Basamaklı HS Kodu,Sektör İsmi
0,2022.0,30910,"Flours, meals and pellets of fish, fit for hum...",3,Fish (03 HS92)
1,2022.0,30990,"Flours, meals and pellets of crustaceans, moll...",3,Fish (03 HS92)
2,2022.0,40320,"Dairy produce: yoghurt, whether or not concent...",4,Diary products (04 HS92)
3,2022.0,41010,Animal products: insects,4,Diary products (04 HS92)
4,2022.0,41090,Animal products: edible products of animal ori...,4,Diary products (04 HS92)
...,...,...,...,...,...
955,2012.0,930110,Military weapons: artillery weapons (e.g. guns...,93,Arms and ammunition (93 HS92)
956,2012.0,930520,"Firearms: parts and accessories, of shotguns o...",93,Arms and ammunition (93 HS92)
957,2012.0,950450,"Games: video game consoles and machines, other...",95,Toys (95 HS92)
958,2012.0,960830,"Pens: fountain, stylograph and other pens",96,Miscellaneous manufactured articles (96 HS92)


In [ ]:
# 5-) 2012'de ve 2017'de yeni eklenen ürünlerin CEE/MENA bölgesinde ticaret hacminde oranları 2012-2021

import pandas as pd
import pandas as pd

# Yıllar ve ilgili HS sürümleri
yillar = list(range(2012, 2022))
hs_versiyonlari = {year: 'HS12' if year <= 2016 else 'HS17' for year in yillar}

# Ülkeler dosyası ve dosya formatı otomatizasyon için
dosya_yolu_format = r"C:\Users\oguzo\Desktop\BACI\BACI_{hs_version}_V202401\BACI_{hs_version}_Y{year}_V202401.csv"
ulkeler_df = pd.read_excel('0-) Ülkeler.xlsx')

# Yeni ürünlerin bulunduğu Excel dosyasını yükle
new_products_df = pd.read_excel('1-) 2012, 2017 ve 2022\'deki yeni ürünlerin detaylı ve genel sektörleri.xlsx')
new_products_df['Yıl'] = new_products_df['Yıl'].ffill()

# 2012 ve 2017'ye ait yeni ürünleri ayır
new_products_2012_df = new_products_df[new_products_df['Yıl'] == 2012]
new_products_2017_df = new_products_df[new_products_df['Yıl'] == 2017]
new_products_2022_df = new_products_df[new_products_df['Yıl'] == 2022]

# Yeni ürünlerin ülkelerdeki ticaret hacim oranlarının hesabı
yeni_urunler_ulke_bazli_ticaret_hacim_oranı = pd.DataFrame()
for index, row in ulkeler_df.iterrows():
    ulke = row['Countries']  
    ulke_kodu = row['country_code']  
    for year in yillar:
        hs_version = hs_versiyonlari[year] 
        dosya_yolu = dosya_yolu_format.format(hs_version=hs_version, year=year)
        df = pd.read_csv(dosya_yolu)
        
        # Yalnızca ilgili ülkeye ait veriyi filtreleme
        ulke_veri = df[df['i'] == ulke_kodu]  
        toplam_ticaret_hacmi = ulke_veri['v'].sum()
        
        # 2012-2016 için new_products_2012_df, 2017-2021 için new_products_2017_df 
        if year <= 2016:
            yeni_urunler_ticaret_hacmi = ulke_veri[ulke_veri['k'].isin(new_products_2012_df['6 Basamaklı HS Kodu'])]['v'].sum()
        else:
            yeni_urunler_ticaret_hacmi = ulke_veri[ulke_veri['k'].isin(new_products_2017_df['6 Basamaklı HS Kodu'])]['v'].sum()
        if toplam_ticaret_hacmi > 0:  
            yeni_urunler_orani = (yeni_urunler_ticaret_hacmi / toplam_ticaret_hacmi) * 100
        else:
            yeni_urunler_orani = 0 
yeni_urunler_ulke_bazli_ticaret_hacim_oranı.at[ulke, year] = yeni_urunler_orani

# Elimizdeki veri panel formatına çevrilmiş şekilde değil onu panel formata çeviriyoruz
yeni_urunler_ulke_bazli_ticaret_hacim_oranı.rename(columns={'Unnamed: 0': 'Ülkeler'}, inplace=True)
yeni_urunler_ulke_bazli_ticaret_hacim_oranı = pd.melt(data, id_vars=['Ülkeler'], var_name='Yıl', value_name='Value')
yeni_urunler_ulke_bazli_ticaret_hacim_oranı = sonuclar3[['Yıl', 'Ülkeler', 'Value']]

# Excele kaydet
yeni_urunler_ulke_bazli_ticaret_hacim_oranı.to_excel("5-) Yeni eklenen ürünlerin seçili ülkelerdeki oranları.xlsx")

In [30]:
# 6-) Seçili yıllarda seçili ülkelerde yeni eklenen ürünlerin HS 2 basamağına göre sektörlerdeki ticaret hacim oranları

import pandas as pd

# Ülkeler dosyası ve dosya formatı otomatizasyon için
ulkeler_df = pd.read_excel('0-) Ülkeler.xlsx')
dosya_yolu_format = r"C:\Users\oguzo\Desktop\BACI\BACI_{hs_version}_V202401\BACI_{hs_version}_Y{year}_V202401.csv"

# Yeni ürünleri okuyoruz ve yılları dolduruyoruz
new_products_df = pd.read_excel('1-) 2012, 2017 ve 2022\'deki yeni ürünlerin detaylı ve genel sektörleri.xlsx')
new_products_df['Yıl'] = new_products_df['Yıl'].ffill()

# 2 basamaklı HS kodlarında tek basamaklı olanlara sıfır ekleme
new_products_df['2 Basamaklı HS Kodu'] = new_products_df['2 Basamaklı HS Kodu'].apply(lambda x: f'{int(x):02d}')

# 2012, 2017, 2022 yıllarındaki yeni ürünler
new_products_2012_df = new_products_df[new_products_df['Yıl'] == 2012]
new_products_2017_df = new_products_df[new_products_df['Yıl'] == 2017]
new_products_2022_df = new_products_df[new_products_df['Yıl'] == 2022]

# Yıllar ve HS versiyonları
yillar = list(range(2012, 2022))
hs_versiyonlari = {year: 'HS12' if year <= 2016 else 'HS17' for year in yillar}


# Her yıl ve ülke için yeni ürünlerdeki sektörlerin ticaret hacim oranlarını hesaplama
sonuclar = []
for year in yillar:
    hs_version = hs_versiyonlari[year]
    dosya_yolu = dosya_yolu_format.format(hs_version=hs_version, year=year)

    # Yılda hangi yeni ürünü kullancağımızı belirle 
    if 2012 <= year <= 2016:
        new_products = new_products_2012_df
    elif 2017 <= year <= 2021:
        new_products = new_products_2017_df
    elif year == 2022:
        new_products = new_products_2022_df

    # Dış ticaret verisini yükle
    df = pd.read_csv(dosya_yolu)

    # HS kodunun sadece ilk 2 basamağını almak için yeni bir sütun oluştur
    df['2_Basamakli_HS'] = df['k'].astype(str).str[:2]

    for index, row in ulkeler_df.iterrows():
        ulke_kodu = row['country_code']
        ulke_adi = row['Countries']

        # Ülkenin toplam ticaret hacmi
        ulke_veri = df[df['i'] == ulke_kodu]
        toplam_ticaret_hacmi = ulke_veri['v'].sum()

        # Yeni ürünlerdeki ticaret hacmi
        yeni_urunler_veri = ulke_veri[ulke_veri['2_Basamakli_HS'].isin(new_products['2 Basamaklı HS Kodu'])]
        yeni_urunler_hacmi = yeni_urunler_veri['v'].sum()

        # Oran
        if toplam_ticaret_hacmi > 0:
            oran = yeni_urunler_hacmi / toplam_ticaret_hacmi
        else:
            oran = 0

        # Sonuçları toplama
        for sektor in new_products['Sektör İsmi'].unique():
            sektor_kodlari = new_products[new_products['Sektör İsmi'] == sektor]['2 Basamaklı HS Kodu']
            sektor_veri = yeni_urunler_veri[yeni_urunler_veri['2_Basamakli_HS'].isin(sektor_kodlari)]
            sektor_hacmi = sektor_veri['v'].sum()

            if toplam_ticaret_hacmi > 0:
                sektor_orani = sektor_hacmi / toplam_ticaret_hacmi
            else:
                sektor_orani = 0
            sonuclar.append([year, ulke_adi, sektor, sektor_orani])
# Sonuçları bir DataFrame'e dönüştürme
sektorel_hacim = pd.DataFrame(sonuclar, columns=['Yıl', 'Ülke', 'Sektör', 'Ticaret Hacmi Oranı'])

# Sonucu excele kaydetme
sektorel_hacim.to_excel("6-) Ülkelerin 2012-2021 arasında yeni eklenen ürünlerde 2 basamaklı HS kodu sektörlerinin ticaret hacim oranı.xlsx")

        Yıl        Ülke                                         Sektör  \
0      2012     Albania                         Live animals (01 HS92)   
1      2012     Albania                                 Meat (02 HS92)   
2      2012     Albania                                 Fish (03 HS92)   
3      2012     Albania                       Diary products (04 HS92)   
4      2012     Albania                               Plants (06 HS92)   
...     ...         ...                                            ...   
15745  2021  Bangladesh                 Industrial Machinery (84 HS92)   
15746  2021  Bangladesh   Electrical machinery and equipment (85 HS92)   
15747  2021  Bangladesh                             Vehicles (87 HS92)   
15748  2021  Bangladesh                            Furniture (94 HS92)   
15749  2021  Bangladesh  Miscellaneous manufactured articles (96 HS92)   

       Ticaret Hacmi Oranı  
0                 0.000000  
1                 0.000000  
2                 0.0000

In [37]:
# 7-) BOZUK SAÇMA SAPAN

import pandas as pd

# Ülkeler dosyası ve dosya formatı otomatizasyon için
ulkeler_df = pd.read_excel('0-) Ülkeler.xlsx')
dosya_yolu_format = r"C:\Users\oguzo\Desktop\BACI\BACI_{hs_version}_V202401\BACI_{hs_version}_Y{year}_V202401.csv"

# Yeni ürünleri okuyoruz ve yılları dolduruyoruz
new_products_df = pd.read_excel('1-) 2012, 2017 ve 2022\'deki yeni ürünlerin detaylı ve genel sektörleri.xlsx')
new_products_df['Yıl'] = new_products_df['Yıl'].ffill()

# 2 basamaklı HS kodlarında olduğu gibi, 6 basamaklı HS kodları üzerinde çalışacağız
new_products_df['6 Basamaklı HS Kodu'] = new_products_df['6 Basamaklı HS Kodu'].astype(str).apply(lambda x: f'{int(x):06d}')

# 2012, 2017, 2022 yıllarındaki yeni ürünler
new_products_2012_df = new_products_df[new_products_df['Yıl'] == 2012]
new_products_2017_df = new_products_df[new_products_df['Yıl'] == 2017]
new_products_2022_df = new_products_df[new_products_df['Yıl'] == 2022]

# Yıllar ve HS versiyonları
yillar = list(range(2012, 2022))
hs_versiyonlari = {year: 'HS12' if year <= 2016 else 'HS17' for year in yillar}

# Her yıl ve ülke için yeni ürünlerdeki sektörlerin ticaret hacim oranlarını hesaplama (6 basamaklı HS kodlarına göre)
sonuclar = []
for year in yillar:
    hs_version = hs_versiyonlari[year]
    dosya_yolu = dosya_yolu_format.format(hs_version=hs_version, year=year)

    # Yılda hangi yeni ürünü kullancağımızı belirle 
    if 2012 <= year <= 2016:
        new_products = new_products_2012_df
    elif 2017 <= year <= 2021:
        new_products = new_products_2017_df
    elif year == 2022:
        new_products = new_products_2022_df

    # Dış ticaret verisini yükle
    df = pd.read_csv(dosya_yolu)

    # HS kodunun 6 basamağını kullanmak için doğrudan alıyoruz
    df['6_Basamakli_HS'] = df['k'].astype(str).apply(lambda x: f'{int(x):06d}')

    for index, row in ulkeler_df.iterrows():
        ulke_kodu = row['country_code']
        ulke_adi = row['Countries']

        # Ülkenin toplam ticaret hacmi
        ulke_veri = df[df['i'] == ulke_kodu]
        toplam_ticaret_hacmi = ulke_veri['v'].sum()

        # Yeni ürünlerdeki ticaret hacmi (6 basamaklı HS kodları ile)
        yeni_urunler_veri = ulke_veri[ulke_veri['6_Basamakli_HS'].isin(new_products['6 Basamaklı HS Kodu'])]
        yeni_urunler_hacmi = yeni_urunler_veri['v'].sum()

        # Oran
        if toplam_ticaret_hacmi > 0:
            oran = yeni_urunler_hacmi / toplam_ticaret_hacmi
        else:
            oran = 0

        # Sonuçları toplama
        for sektor in new_products['Sektör İsmi'].unique():
            sektor_kodlari = new_products[new_products['Sektör İsmi'] == sektor]['6 Basamaklı HS Kodu']
            sektor_veri = yeni_urunler_veri[yeni_urunler_veri['6_Basamakli_HS'].isin(sektor_kodlari)]
            sektor_hacmi = sektor_veri['v'].sum()

            if toplam_ticaret_hacmi > 0:
                sektor_orani = sektor_hacmi / toplam_ticaret_hacmi
            else:
                sektor_orani = 0
            sonuclar.append([year, ulke_adi, sektor, sektor_orani])
# Sonuçları bir DataFrame'e dönüştürme
sektorel_hacim_6 = pd.DataFrame(sonuclar, columns=['Yıl', 'Ülke', 'Sektör', 'Ticaret Hacmi Oranı'])

# Sonucu excele kaydetme
sektorel_hacim_6.to_excel("7-) Ülkelerin 2012-2021 arasında yeni eklenen ürünlerde 6 basamaklı HS kodu sektörlerinin ticaret hacim oranı.xlsx")

In [38]:
sektorel_hacim_6

,Yıl,Ülke,Sektör,Ticaret Hacmi Oranı
0,2012,Albania,Live animals (01 HS92),0.000000e+00
1,2012,Albania,Meat (02 HS92),8.228133e-06
2,2012,Albania,Fish (03 HS92),3.377376e-03
3,2012,Albania,Diary products (04 HS92),3.357923e-03
4,2012,Albania,Plants (06 HS92),1.876497e-04
...,...,...,...,...
15745,2021,Bangladesh,Industrial Machinery (84 HS92),7.235217e-07
15746,2021,Bangladesh,Electrical machinery and equipment (85 HS92),6.983049e-05
15747,2021,Bangladesh,Vehicles (87 HS92),1.103706e-05
15748,2021,Bangladesh,Furniture (94 HS92),5.852479e-06


In [45]:
# 8-) 

import pandas as pd
import numpy as np

# Yüklenen dosyalar
ulkeler_file_path = '0-) Ülkeler.xlsx'
new_products_file_path = '1-) 2012, 2017 ve 2022\'deki yeni ürünlerin detaylı ve genel sektörleri.xlsx'

# Ülkeler dosyasını yükleyelim
ulkeler_df = pd.read_excel(ulkeler_file_path)

# Yeni ürünler dosyasını yükleyelim ve yıl kolonundaki boşlukları dolduralım
new_products_df = pd.read_excel(new_products_file_path)
new_products_df['Yıl'] = new_products_df['Yıl'].ffill()

# 6 basamaklı HS kodları üzerinde çalışacağız
new_products_df['6 Basamaklı HS Kodu'] = new_products_df['6 Basamaklı HS Kodu'].astype(str).apply(lambda x: f'{int(x):06d}')

# 2012, 2017, 2022 yıllarındaki yeni ürünler
new_products_2012_df = new_products_df[new_products_df['Yıl'] == 2012]
new_products_2017_df = new_products_df[new_products_df['Yıl'] == 2017]
new_products_2022_df = new_products_df[new_products_df['Yıl'] == 2022]

# Yıllar ve HS versiyonları
yillar = list(range(2012, 2022))
hs_versiyonlari = {year: 'HS12' if year <= 2016 else 'HS17' for year in yillar}

# RCA hesaplaması için sonuçları tutacak liste
sonuclar = []

# Varsayılan dosya yolu formatı (dış ticaret verileri)
dosya_yolu_format = r"C:/Users/oguzo/Desktop/BACI/BACI_{hs_version}_V202401/BACI_{hs_version}_Y{year}_V202401.csv"

# Her yıl ve ülke için yeni ürünlerdeki sektörlerin ticaret hacim oranlarını hesaplama (6 basamaklı HS kodlarına göre)
for year in yillar:
    hs_version = hs_versiyonlari[year]
    dosya_yolu = dosya_yolu_format.format(hs_version=hs_version, year=year)

    # Yılda hangi yeni ürünü kullancağımızı belirle 
    if 2012 <= year <= 2016:
        new_products = new_products_2012_df
    elif 2017 <= year <= 2021:
        new_products = new_products_2017_df
    elif year == 2022:
        new_products = new_products_2022_df

    # Dış ticaret verisini yükle
    try:
        print(f"{year} yılına ait dosya yükleniyor: {dosya_yolu}")
        df = pd.read_csv(dosya_yolu)

        # Ülke toplam ihracatını bulalım (Her ülke ve yıl için)
        for index, row in ulkeler_df.iterrows():
            ulke_kodu = row['country_code']
            ulke_adi = row['Countries']
            
            # İlgili ülke verisini filtrele
            ulke_veri = df[df['i'] == ulke_kodu]

            # Boş veriler olup olmadığını kontrol edelim
            if ulke_veri.empty:
                print(f"{ulke_adi} için {year} yılına ait veri bulunamadı.")
                continue

            toplam_ticaret_hacmi = ulke_veri['v'].sum()

            # Yeni ürünler için ticaret hacmi
            yeni_urunler_veri = ulke_veri[ulke_veri['k'].isin(new_products['6 Basamaklı HS Kodu'])]
            yeni_urunler_ticaret_hacmi = yeni_urunler_veri['v'].sum()

            # Dünya toplamı (tüm ülkeler için)
            dunya_toplam_ticaret_hacmi = df['v'].sum()
            dunya_yeni_urunler_ticaret_hacmi = df[df['k'].isin(new_products['6 Basamaklı HS Kodu'])]['v'].sum()

            # RCA hesaplaması (kontroller eklenerek)
            if (toplam_ticaret_hacmi > 0 and 
                dunya_toplam_ticaret_hacmi > 0 and 
                yeni_urunler_ticaret_hacmi >= 0 and 
                dunya_yeni_urunler_ticaret_hacmi >= 0):
                
                # Bölme işlemi sırasında sıfır bölme hatası olup olmadığını kontrol ediyoruz
                if dunya_yeni_urunler_ticaret_hacmi == 0 or toplam_ticaret_hacmi == 0:
                    rca = np.nan  # Sıfır bölme durumunda NaN değeri atanır
                else:
                    rca = (yeni_urunler_ticaret_hacmi / toplam_ticaret_hacmi) / (dunya_yeni_urunler_ticaret_hacmi / dunya_toplam_ticaret_hacmi)
            else:
                rca = np.nan  # Geçersiz veya sıfır bölme durumu için NaN değeri atanır
            
            # Sonuçları kaydet
            sonuclar.append({
                'Yıl': year,
                'Ülke': ulke_adi,
                'HS Versiyonu': hs_version,
                'RCA': rca
            })
    except FileNotFoundError:
        print(f"{year} yılına ait dosya bulunamadı: {dosya_yolu}")
        continue

# Sonuçları DataFrame'e çevirelim
sonuclar_df = pd.DataFrame(sonuclar)

# Sonuçları Excel dosyasına kaydedelim
output_file_path = 'RCA_sonuclari_2012_2021.xlsx'
sonuclar_df.to_excel(output_file_path, index=False)

# RCA sonuçlarını gösterelim
print(f"RCA sonuçları kaydedildi: {output_file_path}")


2012 yılına ait dosya yükleniyor: C:/Users/oguzo/Desktop/BACI/BACI_HS12_V202401/BACI_HS12_Y2012_V202401.csv
2013 yılına ait dosya yükleniyor: C:/Users/oguzo/Desktop/BACI/BACI_HS12_V202401/BACI_HS12_Y2013_V202401.csv
2014 yılına ait dosya yükleniyor: C:/Users/oguzo/Desktop/BACI/BACI_HS12_V202401/BACI_HS12_Y2014_V202401.csv
2015 yılına ait dosya yükleniyor: C:/Users/oguzo/Desktop/BACI/BACI_HS12_V202401/BACI_HS12_Y2015_V202401.csv
2016 yılına ait dosya yükleniyor: C:/Users/oguzo/Desktop/BACI/BACI_HS12_V202401/BACI_HS12_Y2016_V202401.csv
2017 yılına ait dosya yükleniyor: C:/Users/oguzo/Desktop/BACI/BACI_HS17_V202401/BACI_HS17_Y2017_V202401.csv
2018 yılına ait dosya yükleniyor: C:/Users/oguzo/Desktop/BACI/BACI_HS17_V202401/BACI_HS17_Y2018_V202401.csv
2019 yılına ait dosya yükleniyor: C:/Users/oguzo/Desktop/BACI/BACI_HS17_V202401/BACI_HS17_Y2019_V202401.csv
2020 yılına ait dosya yükleniyor: C:/Users/oguzo/Desktop/BACI/BACI_HS17_V202401/BACI_HS17_Y2020_V202401.csv
2021 yılına ait dosya yüklen